# Import packages

In [ ]:
import sys
sys.path.insert(0, "../")

from reconstruction.datahandler.datahandler import DataHandler
from reconstruction.segmentation.segmentation import Segmentation
import reconstruction.utils.utils as utils

import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d

# Set variables

In [ ]:
root = "../data/data_10_19"
img_path = "fast_collection_2022_10_19"
calib_path = "calibration_2022_10_19_11_17_26_1666171046886500093"

# Can be None or ["215122254778", "151422251878", "215122256544", "213522251068"]
cams_to_keep = ["215122256544", "215122254778"]

# Load data

In [ ]:
dh_ = DataHandler(root, img_path, calib_path, cams_to_keep)

In [ ]:
person_rgb, person_depth = dh_.load_images("mans")
bg_rgb, bg_depth = dh_.load_images("background")

trans = dh_.load_transformations()
depth_scales = dh_.load_depth_scales()
intrinsics = dh_.load_intrinsics()

In [ ]:
print(depth_scales)

In [ ]:
print(trans)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 8))
ax[0].imshow(person_rgb[cams_to_keep[0]])
ax[1].imshow(bg_rgb[cams_to_keep[0]])

# Segment person

In [ ]:
seg = Segmentation()

In [ ]:
out_rgb, out_dep = seg.segment_person(
    list(person_rgb.values()),
    list(person_depth.values())
)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 8))
ax[0].imshow(out_rgb[0])
ax[1].imshow(out_dep[0])

# Create Point Cloud

In [ ]:
pcd = utils.create_point_cloud(
    dict(zip(person_rgb.keys(), out_rgb)),
    dict(zip(person_depth.keys(), out_dep)),
    depth_scales,
    trans,
    intrinsics)

In [ ]:
o3d.visualization.draw_geometries([pcd])

# Create Mesh

In [ ]:
mesh = utils.create_mesh_tsdf(
    dict(zip(person_rgb.keys(), out_rgb)),
    dict(zip(person_depth.keys(), out_dep)),
    depth_scales,
    trans,
    intrinsics)

In [ ]:
o3d.visualization.draw_geometries([mesh])